In [ ]:
# Bài thực hành 2: Dự đoán giá nhà với Mạng nơ-ron
# ====================================================================================

In [ ]:
# Import các thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# fetch_california_housing: Hàm tải bộ dữ liệu California Housing
from sklearn.datasets import fetch_california_housing
# train_test_split: Hàm chia dữ liệu thành tập huấn luyện và tập kiểm tra
from sklearn.model_selection import train_test_split
# StandardScaler: Công cụ chuẩn hóa dữ liệu
from sklearn.preprocessing import StandardScaler
# Các thành phần từ TensorFlow/Keras để xây dựng mạng nơ-ron
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# Các hàm đánh giá mô hình
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:
# -------------------------------------------------------------
# STEP 1: TẢI DỮ LIỆU CALIFORNIA HOUSING
# -------------------------------------------------------------
print("Step 1: Tải bộ dữ liệu California Housing")

# fetch_california_housing(): Hàm tải bộ dữ liệu California Housing từ scikit-learn
# Bộ dữ liệu này chứa thông tin về giá nhà ở các khu vực của California
data = fetch_california_housing()

# Tách dữ liệu thành đặc trưng (X) và biến mục tiêu (y)
# data.data: Ma trận đặc trưng
# data.target: Vector mục tiêu (giá nhà)
X, y = data.data, data.target

# Chuyển đổi dữ liệu thành DataFrame để dễ xử lý
# data.feature_names: Tên của các đặc trưng
feature_names = data.feature_names
# pd.DataFrame(): Tạo DataFrame từ ma trận đặc trưng và tên cột
df = pd.DataFrame(X, columns=feature_names)
# Thêm cột 'Price' (giá nhà) vào DataFrame
df['Price'] = y

# Hiển thị thông tin về bộ dữ liệu
# df.shape: Kích thước của DataFrame (số hàng, số cột)
print(f"Kích thước dữ liệu: {df.shape}")
print(f"Số lượng mẫu: {df.shape[0]}")
# Trừ đi 1 vì cột cuối cùng là giá nhà (biến mục tiêu)
print(f"Số lượng đặc trưng: {df.shape[1] - 1}")

# In thông tin về các đặc trưng
print("\nThông tin về các đặc trưng:")
for i, name in enumerate(feature_names):
    print(f"{i+1}. {name}")

# Hiển thị 5 mẫu đầu tiên
# df.head(): Lấy 5 hàng đầu tiên của DataFrame
print("\n5 mẫu đầu tiên:")
print(df.head())

# Thống kê mô tả
# df.describe(): Tính toán các thống kê cơ bản (trung bình, độ lệch chuẩn, min, max...)
print("\nThống kê mô tả:")
print(df.describe())


In [ ]:
# -------------------------------------------------------------
# STEP 2: KHÁM PHÁ DỮ LIỆU (EDA - EXPLORATORY DATA ANALYSIS)
# -------------------------------------------------------------
print("\nStep 2: Khám phá dữ liệu")

# Vẽ histogram cho biến mục tiêu (giá nhà)
# plt.figure(): Tạo một figure mới với kích thước 10x6 inch
plt.figure(figsize=(10, 6))
# plt.hist(): Vẽ histogram
# bins=50: Chia dữ liệu thành 50 khoảng
plt.hist(y, bins=50)
plt.xlabel('Giá nhà (đơn vị $100,000)')
plt.ylabel('Số lượng')
plt.title('Phân phối giá nhà ở California')
plt.show()

# Tính toán ma trận tương quan
# df.corr(): Tính ma trận tương quan giữa các cột trong DataFrame
plt.figure(figsize=(12, 8))
correlation_matrix = df.corr()
# sns.heatmap(): Vẽ biểu đồ heatmap cho ma trận tương quan
# annot=True: Hiển thị giá trị trong mỗi ô
# cmap='coolwarm': Sử dụng bảng màu từ xanh (tương quan âm) đến đỏ (tương quan dương)
# fmt=".2f": Định dạng số với 2 chữ số thập phân
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Ma trận tương quan')
plt.show()

# Vẽ các biểu đồ scatter plot để xem mối quan hệ giữa một số đặc trưng và giá nhà
plt.figure(figsize=(15, 10))

# Mối quan hệ giữa thu nhập trung bình và giá nhà
# plt.subplot(2, 2, 1): Chia figure thành 2 hàng, 2 cột và chọn vị trí 1
plt.subplot(2, 2, 1)
# plt.scatter(): Vẽ biểu đồ scatter (phân tán)
# alpha=0.5: Độ trong suốt của điểm (0: trong suốt hoàn toàn, 1: không trong suốt)
plt.scatter(df['MedInc'], df['Price'], alpha=0.5)
plt.xlabel('Thu nhập trung bình')
plt.ylabel('Giá nhà ($100,000)')
plt.title('Thu nhập vs Giá nhà')

# Mối quan hệ giữa tuổi nhà và giá nhà
plt.subplot(2, 2, 2)
plt.scatter(df['HouseAge'], df['Price'], alpha=0.5)
plt.xlabel('Tuổi nhà')
plt.ylabel('Giá nhà ($100,000)')
plt.title('Tuổi nhà vs Giá nhà')

# Mối quan hệ giữa số phòng trung bình và giá nhà
plt.subplot(2, 2, 3)
plt.scatter(df['AveRooms'], df['Price'], alpha=0.5)
plt.xlabel('Số phòng trung bình')
plt.ylabel('Giá nhà ($100,000)')
plt.title('Số phòng vs Giá nhà')

# Mối quan hệ giữa vĩ độ và giá nhà
plt.subplot(2, 2, 4)
plt.scatter(df['Latitude'], df['Price'], alpha=0.5)
plt.xlabel('Vĩ độ')
plt.ylabel('Giá nhà ($100,000)')
plt.title('Vĩ độ vs Giá nhà')

plt.tight_layout()
plt.show()

In [ ]:
# -------------------------------------------------------------
# STEP 3: CHIA TẬP DỮ LIỆU
# -------------------------------------------------------------
print("\nStep 3: Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra")

# train_test_split(): Chia dữ liệu thành tập huấn luyện và tập kiểm tra
# test_size=0.2: 20% dữ liệu dùng cho tập kiểm tra, 80% cho tập huấn luyện
# random_state=42: Giá trị khởi tạo cho bộ sinh số ngẫu nhiên, giúp kết quả có thể tái tạo lại
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Số lượng mẫu huấn luyện: {X_train.shape[0]}")
print(f"Số lượng mẫu kiểm tra: {X_test.shape[0]}")

In [ ]:
# -------------------------------------------------------------
# STEP 4: CHUẨN HÓA DỮ LIỆU
# -------------------------------------------------------------
print("\nStep 4: Chuẩn hóa dữ liệu")

# StandardScaler(): Tạo đối tượng chuẩn hóa dữ liệu
# Chuẩn hóa dữ liệu giúp mô hình hội tụ nhanh hơn và ổn định hơn
scaler = StandardScaler()

# fit_transform(): Học tham số chuẩn hóa từ dữ liệu huấn luyện và áp dụng lên chính nó
# Biến đổi dữ liệu để có trung bình bằng 0 và độ lệch chuẩn bằng 1
X_train = scaler.fit_transform(X_train)

# transform(): Áp dụng tham số chuẩn hóa đã học lên dữ liệu kiểm tra
# Không fit lại trên dữ liệu kiểm tra để tránh data leakage
X_test = scaler.transform(X_test)

# Hiển thị giá trị trung bình và độ lệch chuẩn của dữ liệu sau khi chuẩn hóa
# np.mean(): Tính giá trị trung bình
# axis=0: Tính theo cột
print("Giá trị trung bình sau khi chuẩn hóa:")
print(np.mean(X_train, axis=0))
print("\nĐộ lệch chuẩn sau khi chuẩn hóa:")
print(np.std(X_train, axis=0))

In [ ]:
# -------------------------------------------------------------
# STEP 5: THIẾT KẾ MẠNG NƠ-RON
# -------------------------------------------------------------
print("\nStep 5: Thiết kế mạng nơ-ron")

# Sequential(): Tạo mô hình tuần tự (các layer xếp tuần tự)
model = Sequential([
    # Dense: Lớp kết nối đầy đủ
    # 64: Số lượng nơ-ron trong lớp
    # activation='relu': Hàm kích hoạt ReLU (Rectified Linear Unit)
    # input_shape=(X_train.shape[1],): Kích thước đầu vào là số đặc trưng
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),

    # Lớp ẩn thứ hai với 64 nơ-ron
    Dense(64, activation='relu'),

    # Lớp đầu ra với 1 nơ-ron (dự đoán giá nhà)
    # Không có hàm kích hoạt vì đây là bài toán hồi quy
    Dense(1)
])

# Tóm tắt kiến trúc mô hình
# Hiển thị số lượng tham số (weights và biases) của mỗi layer và tổng số
model.summary()

In [ ]:
# -------------------------------------------------------------
# STEP 6: BIÊN DỊCH VÀ HUẤN LUYỆN MÔ HÌNH
# -------------------------------------------------------------
print("\nStep 6: Biên dịch và huấn luyện mô hình")

# Biên dịch mô hình
# optimizer='adam': Thuật toán tối ưu Adam, phổ biến và hiệu quả
# loss='mse': Hàm mất mát Mean Squared Error, phù hợp cho bài toán hồi quy
# metrics=['mae']: Theo dõi Mean Absolute Error trong quá trình huấn luyện
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [ ]:
# Huấn luyện mô hình
# model.fit(): Hàm huấn luyện mô hình
# X_train, y_train: Dữ liệu huấn luyện và nhãn
# epochs=50: Số lần lặp qua toàn bộ dữ liệu huấn luyện
# batch_size=32: Số mẫu được xử lý trong mỗi bước cập nhật
# validation_split=0.1: Dùng 10% dữ liệu huấn luyện làm tập validation
# verbose=1: Hiển thị thanh tiến trình trong quá trình huấn luyện
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

In [ ]:
# -------------------------------------------------------------
# STEP 7: ĐÁNH GIÁ MÔ HÌNH
# -------------------------------------------------------------
print("\nStep 7: Đánh giá mô hình")

# Đánh giá mô hình trên tập kiểm tra
# model.evaluate(): Tính toán loss và các metrics trên dữ liệu kiểm tra
# verbose=0: Không hiển thị thanh tiến trình
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"Test MAE: {test_mae:.4f}") # Mean Absolute Error
print(f"Test MSE: {test_loss:.4f}") # Mean Squared Error

In [ ]:
# Dự đoán trên tập kiểm tra
# model.predict(): Dự đoán đầu ra cho dữ liệu đầu vào
# flatten(): Chuyển mảng 2D thành mảng 1D
y_pred = model.predict(X_test).flatten()

In [ ]:
# Tính các chỉ số đánh giá
# mean_squared_error(): Tính MSE giữa giá trị thực và dự đoán
mse = mean_squared_error(y_test, y_pred)
# np.sqrt(): Tính căn bậc hai để có RMSE
rmse = np.sqrt(mse)
# mean_absolute_error(): Tính MAE giữa giá trị thực và dự đoán
mae = mean_absolute_error(y_test, y_pred)
# r2_score(): Tính hệ số R² (0-1, càng gần 1 càng tốt)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R-squared (R²): {r2:.4f}")

In [ ]:
# Vẽ đồ thị quá trình huấn luyện
plt.figure(figsize=(12, 4))

# Đồ thị MAE
plt.subplot(1, 2, 1)
# history.history['mae']: MAE trên tập huấn luyện
plt.plot(history.history['mae'], label='Training MAE')
# history.history['val_mae']: MAE trên tập validation
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('MAE over epochs')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()

# Đồ thị mất mát (MSE)
plt.subplot(1, 2, 2)
# history.history['loss']: Mất mát trên tập huấn luyện
plt.plot(history.history['loss'], label='Training Loss')
# history.history['val_loss']: Mất mát trên tập validation
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss (MSE)')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# -------------------------------------------------------------
# STEP 8: TRỰC QUAN HÓA KẾT QUẢ DỰ ĐOÁN
# -------------------------------------------------------------
print("\nStep 8: Trực quan hóa kết quả dự đoán")

# So sánh giá trị thực tế và dự đoán
plt.figure(figsize=(10, 6))
# plt.scatter(): Vẽ biểu đồ scatter (phân tán)
plt.scatter(y_test, y_pred, alpha=0.5)
# Vẽ đường chéo y=x (dự đoán hoàn hảo)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel('Giá thực tế ($100,000)')
plt.ylabel('Giá dự đoán ($100,000)')
plt.title('So sánh giá thực tế và giá dự đoán')
plt.show()

# Hiển thị phân phối của sai số
# errors: Sai số giữa giá trị dự đoán và giá trị thực
errors = y_pred - y_test
plt.figure(figsize=(10, 6))
plt.hist(errors, bins=50)
plt.xlabel('Sai số dự đoán')
plt.ylabel('Số lượng')
plt.title('Phân phối sai số dự đoán')
# Vẽ đường dọc tại x=0
plt.axvline(x=0, color='r', linestyle='--')
plt.show()

In [ ]:
# -------------------------------------------------------------
# STEP 9: THỬ NGHIỆM MÔ HÌNH VỚI CÁC VÍ DỤ CỤ THỂ
# -------------------------------------------------------------
print("\nStep 9: Thử nghiệm mô hình với các ví dụ cụ thể")

# Chọn 5 mẫu ngẫu nhiên từ tập kiểm tra
# np.random.randint(): Tạo số nguyên ngẫu nhiên
# len(X_test): Số lượng mẫu trong tập kiểm tra
sample_indices = np.random.randint(0, len(X_test), 5)
sample_X = X_test[sample_indices]
sample_y_true = y_test[sample_indices]
sample_y_pred = model.predict(sample_X).flatten()

# Tạo bảng so sánh
# pd.DataFrame(): Tạo DataFrame với các cột chỉ định
comparison_df = pd.DataFrame({
    'Giá thực tế': sample_y_true,
    'Giá dự đoán': sample_y_pred,
    'Sai số': sample_y_pred - sample_y_true,
    'Sai số phần trăm': ((sample_y_pred - sample_y_true) / sample_y_true) * 100
})
print("So sánh dự đoán cho 5 mẫu ngẫu nhiên:")
print(comparison_df)


In [ ]:
# -------------------------------------------------------------
# STEP 10: CẢI THIỆN MÔ HÌNH (TÙY CHỌN)
# -------------------------------------------------------------
print("\nStep 10: Thử nghiệm cải thiện mô hình")

# Xây dựng mô hình phức tạp hơn với 3 lớp ẩn
improved_model = Sequential([
    # Lớp ẩn thứ nhất với 128 nơ-ron
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    # Lớp ẩn thứ hai với 64 nơ-ron
    Dense(64, activation='relu'),
    # Lớp ẩn thứ ba với 32 nơ-ron
    Dense(32, activation='relu'),
    # Lớp đầu ra với 1 nơ-ron
    Dense(1)
])

# Biên dịch mô hình cải tiến
improved_model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Huấn luyện mô hình cải tiến
improved_history = improved_model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# Đánh giá mô hình cải tiến
y_pred_improved = improved_model.predict(X_test).flatten()
improved_mse = mean_squared_error(y_test, y_pred_improved)
improved_rmse = np.sqrt(improved_mse)
improved_mae = mean_absolute_error(y_test, y_pred_improved)
improved_r2 = r2_score(y_test, y_pred_improved)

print("\nKết quả mô hình cải tiến:")
print(f"MSE: {improved_mse:.4f}")
print(f"RMSE: {improved_rmse:.4f}")
print(f"MAE: {improved_mae:.4f}")
print(f"R²: {improved_r2:.4f}")

# So sánh hai mô hình
print("\nSo sánh hai mô hình:")
comparison = pd.DataFrame({
    'Mô hình ban đầu': [mse, rmse, mae, r2],
    'Mô hình cải tiến': [improved_mse, improved_rmse, improved_mae, improved_r2]
}, index=['MSE', 'RMSE', 'MAE', 'R²'])
print(comparison)